In [1]:
# Function for generating histogram
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mlt
import datetime 
import import_ipynb
import histplot

importing Jupyter notebook from histplot.ipynb
644
3122


In [2]:
ASC = '__ASC_ASCIIIn.mrk'
direct = 'C:/Users/tyron/Documents/Geoscience/weather/'

In [3]:
# Summer 2018 as an example
Summer18 = histplot.getData(18, direct, ASC, 152, 92, 6 , float)


In [4]:
def calc(precip_data, what_bin):
    hist, bin_edge = np.histogram(precip_data, bins = what_bin)
    log_y = np.log(hist)
    bin_val = np.zeros(len(what_bin)-1)
    for i in range(len(what_bin)-1):
        bin_val[i] = (what_bin[i] + what_bin[i+1])/2
    logx = np.log(bin_val)
    curve_fit = np.polyfit(logx, log_y,1)
    curve_fit[1] = np.exp(curve_fit[1])
    return curve_fit, bin_val

In [5]:
# Exponentials get from durations
def exponential(in_put, percent, exclude):
    first_bin = np.zeros(100/percent)
    for i in range(0, 100/percent):    
        first_bin[i] = np.percentile(in_put, percent*i)
    second_bin = np.unique(first_bin)
    stop1 = np.array([exclude])
    final_bin = np.setdiff1d(second_bin, stop1)
    curvefit, bin_vals = calc(in_put, final_bin)
    return curvefit, bin_vals
    

In [ ]:
def model_1(data, amount, howmuch):
    precip, nonprecip, preciptotal, totalprep = histplot.preciphist(data, len(data), amount, 1, 1)
    count = 0
    for i in range(0, len(data)):
         hold = precip17[i]
            if hold > 0:
                count += 1
    # Calculate intensities 
    inten = np.divide(preciptotal[:count], precip[:count])
    
    
    # Exponentials for precipitation duration 

    # Exponentials for precipitation duration 

    # Exponentials for Intensity of Precipitation duration
    
    
    for i in range(howmuch):
        minutes = 0;
        
        while (len(data) > minutes):
            # run probability of precipitation event duration
            pre = np.random.choice()
            # run probability of precipitation event intensity
            inten = np.random.choice()
            # run probability of non-precipitation event duration
            nonpre = np.random.choice()
            
